### **1. Importing Liabraries**

In [39]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import resample

### **2. Loading Datasets**


In [28]:
data=pd.read_csv("../data/clean_data.csv",index_col=0,parse_dates=['trans_date_trans_time','dob'])
data.head(10)
print("Dataset has " , data.shape[0] , " rows and ",data.shape[1] , " columns")
print('column names: \n') 
print('\n'.join(list(data.columns)))

Dataset has  1852394  rows and  29  columns
column names: 

trans_date_trans_time
cc_num
merchant
category
amt
first
last
gender
street
city
state
zip
lat
long
city_pop
job
dob
trans_num
unix_time
merch_lat
merch_long
is_fraud
label
age
trans_year
trans_month
trans_days
trans_week_days
trans_hour


### **3. Deleting Unnecessary Columns**

In [29]:
# list of the columns to be dropped

drop_cols = ['trans_date_trans_time','street','merchant','zip','first','last','trans_num','job','dob','city','trans_days','trans_year']

In [30]:
data.drop(drop_cols, axis = 1, inplace = True)
data.reset_index(drop=True, inplace = True)
list(data.columns) # lets look at the remaining list of columns


['cc_num',
 'category',
 'amt',
 'gender',
 'state',
 'lat',
 'long',
 'city_pop',
 'unix_time',
 'merch_lat',
 'merch_long',
 'is_fraud',
 'label',
 'age',
 'trans_month',
 'trans_week_days',
 'trans_hour']

In [43]:
data

,cc_num,category,amt,gender,state,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,label,age,trans_month,trans_week_days,trans_hour
0,2703186189652095,misc_net,4.97,F,NC,36.0788,-81.1781,3495,1325376018,36.011293,-82.048315,0,Train,30.0,1,Tuesday,0
1,630423337322,grocery_pos,107.23,F,WA,48.8878,-118.2105,149,1325376044,49.159047,-118.186462,0,Train,40.0,1,Tuesday,0
2,38859492057661,entertainment,220.11,M,ID,42.1808,-112.2620,4154,1325376051,43.150704,-112.154481,0,Train,56.0,1,Tuesday,0
3,3534093764340240,gas_transport,45.00,M,MT,46.2306,-112.1138,1939,1325376076,47.034331,-112.561071,0,Train,51.0,1,Tuesday,0
4,375534208663984,misc_pos,41.96,M,VA,38.4207,-79.4629,99,1325376186,38.674999,-78.632459,0,Train,32.0,1,Tuesday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,30560609640617,health_fitness,43.77,M,MO,40.4931,-91.8912,519,1388534347,39.946837,-91.333331,0,Test,54.0,12,Thursday,23
1852390,3556613125071656,kids_pets,111.84,M,TX,29.0393,-95.4401,28739,1388534349,29.661049,-96.186633,0,Test,21.0,12,Thursday,23
1852391,6011724471098086,kids_pets,86.88,F,WA,46.1966,-118.9017,3684,1388534355,46.658340,-119.715054,0,Test,39.0,12,Thursday,23
1852392,4079773899158,travel,7.99,M,ID,44.6255,-116.4493,129,1388534364,44.470525,-117.080888,0,Test,55.0,12,Thursday,23


### **4. Split the Train - Test Dataset**

In [44]:
train_data = data[data['label'] == 'Train'] # train in label

test_data = data[data['label'] == 'Test'] # test in label


In [45]:
train_data.drop('label', axis =1, inplace = True) # dropping the label column

test_data.drop('label', axis =1, inplace = True) # dropping the label column

In [46]:
X_train = train_data.drop('is_fraud', axis = 1) # features for training
y_train = train_data['is_fraud'] # label for training

X_test = test_data.drop('is_fraud', axis = 1) # features for testing
y_test = test_data['is_fraud'] # lables for testing

In [47]:
train_majority = train_data[train_data['is_fraud'] == 0] # create a subset of majority class

train_minority = train_data[train_data['is_fraud'] == 1] # create a subset of minority class

majority_count = len(train_majority) # lets count the number of records in the majority class

majority_count # dispaly the count

1289169

In [48]:
minority_resampled = resample(train_minority,n_samples = majority_count) # resampling the minority class using the boostaping method makes it equavalent in count with the majority class

In [49]:
train_symmetric = pd.concat([train_majority,minority_resampled], axis = 0) # concatinating the majority class subset and resampled minority class dataset to create a symmetric dataset

train_symmetric['is_fraud'].value_counts(normalize = True) # lets confirm the train symmetric dataset possess the equal distribution of classes.

0    0.5
1    0.5
Name: is_fraud, dtype: float64